# Intake, Leaflet, ...

Also check out this repo!
https://github.com/jupyter-widgets/ipyleaflet


In case we don't have a proper leaflet integration... try some of the following.

In [1]:
import intake
from pathlib import Path

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
def f(x):
    return x

In [4]:
interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

In [5]:
import requests
import os
import warnings
from tqdm.auto import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray_leaflet
from ipyleaflet import Map, basemaps, LayersControl, WidgetControl
from ipywidgets import FloatSlider
import matplotlib.pyplot as plt

In [6]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/as_dem_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = os.path.join(name, name, 'w001001.adf')

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

In [7]:
da = rioxarray.open_rasterio(adffile, masked=True)
da

<xarray.DataArray (band: 1, y: 8400, x: 15000)>
[126000000 values with dtype=float64]
Coordinates:
  * band         (band) int64 1
  * y            (y) float64 60.0 59.99 59.98 59.97 ... -9.979 -9.987 -9.996
  * x            (x) float64 55.0 55.01 55.02 55.03 ... 180.0 180.0 180.0 180.0
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    grid_mapping:  spatial_ref

In [8]:
da = da.sel(band=1)
da.name = 'DEM'

In [9]:
da = da.chunk((1000, 1000))
warnings.filterwarnings("ignore")

In [10]:
m = Map(center=[40, 115], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

Map(center=[40, 115], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [11]:
l = da.leaflet.plot(m, colormap=plt.cm.terrain)
# l.interact(opacity=(0., 1.))

In [12]:
layers_control = LayersControl(position='topright')
m.add_control(layers_control)

opacity_slider = FloatSlider(description='Opacity:', min=0, max=1, value=1)

def set_opacity(change):
    l.opacity = change['new']

opacity_slider.observe(set_opacity, names='value')
slider_control = WidgetControl(widget=opacity_slider, position='bottomleft')
m.add_control(slider_control)

In [13]:
da.leaflet.select()

In [14]:
da_selected = da.leaflet.get_selection()

In [16]:
if da_selected is not None:
    print(da_selected)

In [17]:
import ipyleaflet 
import json 
import pandas as pd
from ipywidgets import link, FloatSlider
from branca.colormap import linear
import random

geo_json_data = json.load(open('us-states.json')) 
m = ipyleaflet.Map(center = (43,-100), zoom = 4)
unemployment = pd.read_csv('US_Unemployment_Oct2012.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'us-states.json'

In [25]:
cat = intake.open_catalog(str(Path.cwd() / 'data' / 'catalog.yml'))
list(cat)

['soil', 'admin', 'elevation']

In [30]:
cat.soil().read()

<xarray.Dataset>
Dimensions:  (lat: 300, lev: 5, lon: 720)
Coordinates:
  * lon      (lon) float64 -179.8 -179.3 -178.8 -178.3 ... 178.7 179.2 179.7
  * lev      (lev) int64 1 2 3 4 5
  * lat      (lat) float64 -59.75 -59.25 -58.75 -58.25 ... 88.75 89.25 89.75
Data variables:
    SUID     (lat, lon) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    TotDep   (lat, lon) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    TopDep   (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    BotDep   (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    ALSA     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    BSAT     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    BULK     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    CECS     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    CECc     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    CFRAG    (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    CLPC     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    CNrt     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    ECEC     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    ELCO     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    ESP      (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    GYPS     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    PHAQ     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    PROP1    (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    SDTO     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    STPC     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    TAWC     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    TCEQ     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    TOTC     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan
    TOTN     (lev, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan

In [8]:
  hydrosheds:
    name: 'HYDROSHEDS'
    description: 'For xarray-leaflet example'
    driver: rasterio
    args:
      urlpath: 'simplecache::https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/as_dem_30s_grid.zip'
      storageoptions:
        simplecache:
          cache_storage: '.cache'
          same_names: true
      chunks: {}

AttributeError: soil

In [9]:
Path.cwd()

PosixPath('/Users/werner-ch/Documents/Repos/xarray-101')

In [5]:
! pip freeze | grep netcdf4

netCDF4 @ file:///Users/runner/miniforge3/conda-bld/netcdf4_1602508558073/work


In [22]:
intake.__version__

'0.6.0'